In [ ]:
pip install faiss-cpu sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 74.3 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.nn import functional as F

In [ ]:
# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Adjust tokenizer for GPT-2
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to eos token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
import PyPDF2
def pdf_to_text(pdf_path, output_txt):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    with open(output_txt, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [ ]:
pdf_to_text("/content/Anxious People.pdf", "/content/Anxious People.txt")
print("PDF scraped")

PDF scraped


In [ ]:
# Custom Dataset for Text Generation
class TextGenerationDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=128):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        print(f"Total characters in text: {len(text)}")
        # Tokenize the entire text
        tokens = tokenizer(text, return_tensors="pt")["input_ids"].squeeze()
        print(f"Total tokens: {len(tokens)}")
        # Split tokens into chunks of `block_size`
        self.examples = [
            tokens[i : i + block_size]
            for i in range(0, len(tokens) - block_size + 1, block_size)
        ]
        print(f"Total examples: {len(self.examples)}")
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

In [ ]:
# Paths to dataset files
train_file_path = "/content/Anxious People.txt"

# Load datasets
dataset = TextGenerationDataset(train_file_path, tokenizer)

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print(len(dataset))
print(len(train_dataset), len(test_dataset))

Total characters in text: 540333


Token indices sequence length is longer than the specified maximum sequence length for this model (150926 > 1024). Running this sequence through the model will result in indexing errors


Total tokens: 150926
Total examples: 1179
1179
943 236


In [ ]:
# DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

In [ ]:
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training Loop
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    batch_cnt = 0
    for batch in train_loader:
        batch_cnt += 1
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=batch, labels=batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        print("Running Batch:", batch_cnt)

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")


Running Batch: 1
Running Batch: 2
Running Batch: 3
Running Batch: 4
Running Batch: 5
Running Batch: 6
Running Batch: 7
Running Batch: 8
Running Batch: 9
Running Batch: 10
Running Batch: 11
Running Batch: 12
Running Batch: 13
Running Batch: 14
Running Batch: 15
Running Batch: 16
Running Batch: 17
Running Batch: 18
Running Batch: 19
Running Batch: 20
Running Batch: 21
Running Batch: 22
Running Batch: 23
Running Batch: 24
Running Batch: 25
Running Batch: 26
Running Batch: 27
Running Batch: 28
Running Batch: 29
Running Batch: 30
Running Batch: 31
Running Batch: 32
Running Batch: 33
Running Batch: 34
Running Batch: 35
Running Batch: 36
Running Batch: 37
Running Batch: 38
Running Batch: 39
Running Batch: 40
Running Batch: 41
Running Batch: 42
Running Batch: 43
Running Batch: 44
Running Batch: 45
Running Batch: 46
Running Batch: 47
Running Batch: 48
Running Batch: 49
Running Batch: 50
Running Batch: 51
Running Batch: 52
Running Batch: 53
Running Batch: 54
Running Batch: 55
Running Batch: 56
R

In [11]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2_v2")
tokenizer.save_pretrained("fine_tuned_gpt2_v2")
print("Fine-tuned model saved.")

Fine-tuned model saved.


In [12]:
# Evaluation Loop
model.eval()
with torch.no_grad():
    eval_loss = 0
    batch_cnt = 0
    for batch in test_loader:
        batch_cnt += 1
        batch = batch.to(device)
        outputs = model(input_ids=batch, labels=batch)
        loss = outputs.loss
        eval_loss += loss.item()
        print("Running Batch:", batch_cnt)

    avg_eval_loss = eval_loss / len(test_loader)
    print(f"Evaluation Loss: {avg_eval_loss:.4f}")

Running Batch: 1
Running Batch: 2
Running Batch: 3
Running Batch: 4
Running Batch: 5
Running Batch: 6
Running Batch: 7
Running Batch: 8
Running Batch: 9
Running Batch: 10
Running Batch: 11
Running Batch: 12
Running Batch: 13
Running Batch: 14
Running Batch: 15
Running Batch: 16
Running Batch: 17
Running Batch: 18
Running Batch: 19
Running Batch: 20
Running Batch: 21
Running Batch: 22
Running Batch: 23
Running Batch: 24
Running Batch: 25
Running Batch: 26
Running Batch: 27
Running Batch: 28
Running Batch: 29
Running Batch: 30
Running Batch: 31
Running Batch: 32
Running Batch: 33
Running Batch: 34
Running Batch: 35
Running Batch: 36
Running Batch: 37
Running Batch: 38
Running Batch: 39
Running Batch: 40
Running Batch: 41
Running Batch: 42
Running Batch: 43
Running Batch: 44
Running Batch: 45
Running Batch: 46
Running Batch: 47
Running Batch: 48
Running Batch: 49
Running Batch: 50
Running Batch: 51
Running Batch: 52
Running Batch: 53
Running Batch: 54
Running Batch: 55
Running Batch: 56
R

In [13]:
# Function to generate text with fine-tuned model
def generate_text(prompt, model, tokenizer, max_length=350, temperature=0.7, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        do_sample=True,  # Enable sampling to allow creativity in the output
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [18]:
print("\nEnter a prompt for text generation:")
user_prompt = input().strip()

generated_text_user = generate_text(user_prompt, model, tokenizer)
print(f"\nGenerated Text for your prompt:\n{generated_text_user}")


Enter a prompt for text generation:
You are an expert on 'Anxious People'. Answer questions clearly.\n\nQ: What do you think about zara?\nA:


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generated Text for your prompt:
You are an expert on 'Anxious People'. Answer questions clearly.\n\nQ: What do you think about zara?\nA: “Maybe she’s a bit of a… I don’t know…”
“Are you upset?”
“I’m upset.”
“I was just surprised.”
“What?”
“Like I said before: “You’re an expert on “Anxious People.”
“Really?”
“Exactly .”
“Are you upset?”
“I was just surprised.”
“Really?”
“No.”
“How angry are you?”
“No I was just surprised.”
“Are you angry?”
“No I was just surprised.”
“Are you angry?”
“No I was just surprised.”
“You’re an expert on “Anxious People.”
“Oh,”
“Exactly .”
“I was just surprised.”
“What?”
“Oh,”
“You should read this!”
“Yes I should read this!”
“What do you mean?”
“I mean: I’m upset.”
“You’re an expert on “Anxious People.”
“I don’ t
